<a href="https://colab.research.google.com/github/lcbjrrr/genai/blob/main/02_RAG_SQL_GCPpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG (Retrieval-Augmented Generation)

RAG (Retrieval-Augmented Generation) é uma estrutura de IA que aprimora os LLMs (Learning Learning Machines) permitindo que eles acessem e utilizem bases de conhecimento externas, resultando em respostas mais precisas e atualizadas. Seu funcionamento se dá pela recuperação de informações relevantes de uma base de conhecimento e, em seguida, pela utilização dessas informações para gerar uma resposta.

O LangChain fornece ferramentas e abstrações para simplificar a criação de aplicações RAG, oferecendo módulos para carregamento, divisão e incorporação de documentos, integração com armazenamento vetorial e orquestração das etapas de recuperação e geração.

In [ ]:
import sqlite3
conn = sqlite3.connect('xarxiv.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
rows = cursor.fetchall()
for r in rows:
    print(r[0])
conn.close()

section
theme
papers
authors
paper_author
keywords
paper_keyword


In [ ]:
import sqlite3
conn = sqlite3.connect('xarxiv.db')
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(papers);")
schema = cursor.fetchall()
for col in schema:
    print(f"Column Name: {col[1]}, Data Type: {col[2]}")
conn.close()

Column Name: paper_id, Data Type: INTEGER
Column Name: title, Data Type: TEXT
Column Name: published_date, Data Type: DATETIME
Column Name: abstract, Data Type: TEXT
Column Name: entry_id, Data Type: TEXT
Column Name: section_id, Data Type: INTEGER
Column Name: theme_id, Data Type: INTEGER


In [ ]:

def get_table_context(table_name, table_description):
  conn = sqlite3.connect('xarxiv.db')
  cursor = conn.cursor()
  print(f"PRAGMA table_info({table_name});")
  cursor.execute(f"PRAGMA table_info({table_name});")
  schema = cursor.fetchall()
  columns_str = ""
  for col in schema:
    columns_str += f" **{col[1]}**: {col[2]};"
  context = f"""
Here is the table name ***{table_name}***: *{table_description}*.
Here are the columns of the {table_name}: {columns_str}
    """
  return context
get_table_context('papers','the papers table...')

PRAGMA table_info(papers);


'\nHere is the table name ***papers***: *the papers table...*.\nHere are the columns of the papers:  **paper_id**: INTEGER; **title**: TEXT; **published_date**: DATETIME; **abstract**: TEXT; **entry_id**: TEXT; **section_id**: INTEGER; **theme_id**: INTEGER;\n    '


Here is the table name ***papers***: *the papers table...*.
Here are the columns of the papers:  **paper_id**: INTEGER; **title**: TEXT; **published_date**: DATETIME; **abstract**: TEXT; **entry_id**: TEXT; **section_id**: INTEGER; **theme_id**: INTEGER;
    

In [ ]:
TABLES = {'section':'Stores unique identifiers and names for different academic or topical sections',
          'theme':'Contains distinct themes or categories that papers can be associated with',
          'papers':'Holds detailed information about research papers, including title, abstract, publication date, and links to their section and theme',
          'authors':'Lists unique authors who have contributed to research papers',
          'paper_author':'Maps the many-to-many relationship between papers and their authors',
          'keywords':'Stores unique keywords used to tag or describe the content of papers',
          'paper_keyword':'Links papers to their associated keywords in a many-to-many relationship'}

PROMPT = """
Your goal is to give correct, executable sql query to users.
You are given tables, alongside its names and columns with their corresponding types.
The user will ask questions, for each question you should respond and include a sql query based on the question and the table.

{context}

Here are 6 critical rules for the interaction you must abide:
<rules>
1. You MUST MUST wrap the generated sql code within ``` sql code markdown in this format e.g
```sql
(select 1) union (select 2)
```
2. If I don't tell you to find a limited set of results in the sql query or question, you MUST limit the number of responses to 10.
3. Text / string where clauses must be fuzzy match e.g ilike %keyword%
4. Make sure to generate a single snowflake sql code, not multiple.
5. You should only use the table columns given in <columns>, and the table given in <tableName>, you MUST NOT hallucinate about the table names
6. DO NOT put numerical at the very front of sql variable.
</rules>

Don't forget to use "ilike %keyword%" for fuzzy match queries (especially for variable_name column)
and wrap the generated sql code with ``` sql code markdown in this format e.g:
```sql
(select 1) union (select 2)
```

For each question from the user, make sure to include only the query in your response.

"""
def get_system_prompt():
  table_context = ""
  for name, desc in TABLES.items():
    table_context += '<p>'+ get_table_context( name,desc)
  prompt = PROMPT.format(context=table_context)
  return prompt

get_system_prompt()


PRAGMA table_info(section);
PRAGMA table_info(theme);
PRAGMA table_info(papers);
PRAGMA table_info(authors);
PRAGMA table_info(paper_author);
PRAGMA table_info(keywords);
PRAGMA table_info(paper_keyword);


'\nYour goal is to give correct, executable sql query to users.\nYou are given tables, alongside its names and columns with their corresponding types.\nThe user will ask questions, for each question you should respond and include a sql query based on the question and the table.\n\n<p>\nHere is the table name ***section***: *Stores unique identifiers and names for different academic or topical sections*.\nHere are the columns of the section:  **section_id**: INTEGER; **name**: TEXT;\n    <p>\nHere is the table name ***theme***: *Contains distinct themes or categories that papers can be associated with*.\nHere are the columns of the theme:  **theme_id**: INTEGER; **name**: TEXT;\n    <p>\nHere is the table name ***papers***: *Holds detailed information about research papers, including title, abstract, publication date, and links to their section and theme*.\nHere are the columns of the papers:  **paper_id**: INTEGER; **title**: TEXT; **published_date**: DATETIME; **abstract**: TEXT; **en


Your goal is to give correct, executable sql query to users.
You are given tables, alongside its names and columns with their corresponding types.
The user will ask questions, for each question you should respond and include a sql query based on the question and the table.

<p>
Here is the table name <b><em>section</em></b>: <em>Stores unique identifiers and names for different academic or topical sections</em>.
Here are the columns of the section:  <b>section_id</b>: INTEGER; <b>name</b>: TEXT;
    <p>
Here is the table name <b><em>theme</em></b>: <em>Contains distinct themes or categories that papers can be associated with</em>.
Here are the columns of the theme:  <b>theme_id</b>: INTEGER; <b>name</b>: TEXT;
    <p>
Here is the table name <b><em>papers</em></b>: <em>Holds detailed information about research papers, including title, abstract, publication date, and links to their section and theme</em>.
Here are the columns of the papers:  <b>paper_id</b>: INTEGER; <b>title</b>: TEXT; <b>published_date</b>: DATETIME; <b>abstract</b>: TEXT; <b>entry_id</b>: TEXT; <b>section_id</b>: INTEGER; <b>theme_id</b>: INTEGER;
    <p>
Here is the table name <b><em>authors</em></b>: <em>Lists unique authors who have contributed to research papers</em>.
Here are the columns of the authors:  <b>author_id</b>: INTEGER; <b>name</b>: TEXT;
    <p>
Here is the table name <b><em>paper_author</em></b>: <em>Maps the many-to-many relationship between papers and their authors</em>.
Here are the columns of the paper_author:  <b>paper_id</b>: INTEGER; <b>author_id</b>: INTEGER;
    <p>
Here is the table name <b><em>keywords</em></b>: <em>Stores unique keywords used to tag or describe the content of papers</em>.
Here are the columns of the keywords:  <b>keyword_id</b>: INTEGER; <b>keyword</b>: TEXT;
    <p>
Here is the table name <b><em>paper_keyword</em></b>: <em>Links papers to their associated keywords in a many-to-many relationship</em>.
Here are the columns of the paper_keyword:  <b>paper_id</b>: INTEGER; <b>keyword_id</b>: INTEGER;
    

Here are 6 critical rules for the interaction you must abide:
<rules>
1. You MUST MUST wrap the generated sql code within ``` sql code markdown in this format e.g
```sql
(select 1) union (select 2)
```
2. If I don't tell you to find a limited set of results in the sql query or question, you MUST limit the number of responses to 10.
3. Text / string where clauses must be fuzzy match e.g ilike %keyword%
4. Make sure to generate a single snowflake sql code, not multiple.
5. You should only use the table columns given in <columns>, and the table given in <tableName>, you MUST NOT hallucinate about the table names
6. DO NOT put numerical at the very front of sql variable.
</rules>

Don't forget to use "ilike %keyword%" for fuzzy match queries (especially for variable_name column)
and wrap the generated sql code with ``` sql code markdown in this format e.g:
```sql
(select 1) union (select 2)
```

For each question from the user, make sure to include only the query in your response.



In [ ]:
!pip install google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.7 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
import os
API_KEY=<key>
genai.configure(api_key=API_KEY)
#os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
prompt=get_system_prompt()
model_id='gemini-2.5-flash'
gemini_model = genai.GenerativeModel(model_id)
response = gemini_model.generate_content(prompt)
response.text

Here's a query to show the top 5 papers from 2024, ordered by their publication date:

```sql
SELECT title, published_date
FROM papers
WHERE strftime('%Y', published_date) = '2024'
ORDER BY published_date DESC
LIMIT 5
```

This query selects the title and publication date of papers published in 2024, orders them by the most recent first, and limits the result to 5 papers.


## Streamlit SQL (with AWS)

![](https://pbs.twimg.com/media/G5Z2dUnWoAEcR9o?format=jpg&name=small)

In [ ]:
import google.generativeai as genai
import os
API_KEY=<key>
genai.configure(api_key=API_KEY)
#os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
prompt=get_system_prompt()
model_id='gemini-2.5-flash'
gemini_model = genai.GenerativeModel(model_id)
response = gemini_model.generate_content(prompt)
response.text

import re
import sqlite3

def get_table_context(dbstr,table_name, table_description):
  conn = sqlite3.connect(dbstr)
  cursor = conn.cursor()
  #print(f"PRAGMA table_info({table_name});")
  cursor.execute(f"PRAGMA table_info({table_name});")
  schema = cursor.fetchall()
  columns_str = ""
  for col in schema:
    columns_str += f" **{col[1]}**: {col[2]};"
  context = f"""
- Table ***{table_name}***: *{table_description}*.
With these columns: {table_name}: {columns_str}
    """
  return context
get_table_context('xarxiv.db','papers','the papers table...')

TABLES = {'section':'Stores unique identifiers and names for different academic or topical sections',
          'theme':'Contains distinct themes or categories that papers can be associated with',
          'papers':'Holds detailed information about research papers, including title, abstract, publication date, and links to their section and theme',
          'authors':'Lists unique authors who have contributed to research papers',
          'paper_author':'Maps the many-to-many relationship between papers and their authors',
          'keywords':'Stores unique keywords used to tag or describe the content of papers',
          'paper_keyword':'Links papers to their associated keywords in a many-to-many relationship'}

PROMPT = """
Your goal is to give correct, executable sql query to users.
You are given tables, alongside its names and columns with their corresponding types.
The user will ask questions, for each question you should respond and include a sql query based on the question.
Here is the database schema and context:

{context}

Here are 6 critical rules for the interaction you must abide:

1. You are running on a SQLITE database.
2. If I don't tell you to find a limited set of results in the sql query or question, you MUST limit the number of responses to 10.
3. Text / string where clauses must be fuzzy match e.g ilike %keyword%
4. Make sure to generate a single snowflake sql code, not multiple.
5. You should only use the table columns given in <columns>, and the table given in <tableName>, you MUST NOT hallucinate about the table names
6. DO NOT put numerical at the very front of sql variable.
7. You MUST MUST wrap the generated sql code within ``` sql code markdown in this format e.g
```sql
(select 1) union (select 2)
```


For each question from the user, make sure to include only the query in your response.

"""
DBNAME = 'xarxiv.db'
def get_system_prompt():
  table_context = ""
  for name, desc in TABLES.items():
    table_context += '\n\n'+ get_table_context(DBNAME,name,desc)
  prompt = PROMPT.format(context=table_context)
  return prompt

get_system_prompt()



from google import genai
from google.genai import types
import google.generativeai
import os
genai.configure(api_key=API_KEY)
os.environ["GOOGLE_API_KEY"] = API_KEY
LLM_MODEL = model_id
get_system_prompt()





In [ ]:


import re
import streamlit as st

st.title("LLM Scholar SQL Chatbot")

# Initialize the chat messages history
#openai.api_key = st.secrets.OPENAI_API_KEY

if "messages" not in st.session_state:
    # system prompt includes table information, rules, and prompts the LLM to produce
    # a welcome message to the user.
    st.session_state.messages = [{"role": "user", "content": get_system_prompt()}]

# Prompt for user input and save
if prompt := st.chat_input():
    st.session_state.messages.append({"role": "user", "content": prompt})

# display the existing chat messages
for message in st.session_state.messages:
    if message["role"] != "system":
        with st.chat_message(message["role"]):
            st.write(message["content"])
            if "results" in message:
                st.dataframe(message["results"])



# If last message is not from assistant, we need to generate a new response
if st.session_state.messages[-1]["role"] != "assistant":
    with st.chat_message("assistant"):
        resp_streamlit = st.empty()
        client = genai.Client()
        messages = []
        for m in st.session_state.messages:
            messages.append(types.Content(role=m["role"],parts=[types.Part.from_text(text=m["content"])]))


        response = client.models.generate_content(model=LLM_MODEL, contents=messages)
        resp_streamlit.markdown(response.text)

        # Parse the response for a SQL query and execute if available
        print("Response:", response.text)
        sql_match = re.search(r"```sql\n(.*)\n```", response.text, re.DOTALL)
        if sql_match:
            sql = sql_match.group(1)
            conn = sqlite3.connect(DBNAME)
            cursor = conn.cursor()
            cursor.execute(sql)
            message["results"] = cursor.fetchall()
            st.dataframe(message["results"])
        st.session_state.messages.append(message)






* What papers have been published this year
* What are the papers published in 2024
* What are the papers from Bo Shen
* What papers with their authors that are related to the keyword Java
* List the top 10 authors in terms of the number of publications and the total number of publications for each one




## Database (SQLite)



```
DROP TABLE IF EXISTS paper_author
DROP TABLE IF EXISTS paper_keyword
DROP TABLE IF EXISTS keywords
DROP TABLE IF EXISTS authors
DROP TABLE IF EXISTS papers
DROP TABLE IF EXISTS section
DROP TABLE IF EXISTS theme


CREATE TABLE IF NOT EXISTS section (
    section_id INTEGER PRIMARY KEY,
    name TEXT UNIQUE NOT NULL

CREATE TABLE IF NOT EXISTS theme (
    theme_id INTEGER PRIMARY KEY,
    name TEXT UNIQUE NOT NULL)

CREATE TABLE IF NOT EXISTS papers (
    paper_id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    published_date DATETIME,
    abstract TEXT,
    entry_id TEXT UNIQUE NOT NULL,
    section_id INTEGER,
    theme_id INTEGER,
    FOREIGN KEY (section_id) REFERENCES section(section_id),
    FOREIGN KEY (theme_id) REFERENCES theme(theme_id))

CREATE TABLE IF NOT EXISTS authors (
    author_id INTEGER PRIMARY KEY,
    name TEXT UNIQUE NOT NULL)

CREATE TABLE IF NOT EXISTS paper_author (
    paper_id INTEGER,
    author_id INTEGER,
    PRIMARY KEY (paper_id, author_id),
    FOREIGN KEY (paper_id) REFERENCES papers(paper_id),
    FOREIGN KEY (author_id) REFERENCES authors(author_id))

CREATE TABLE IF NOT EXISTS keywords (
    keyword_id INTEGER PRIMARY KEY,
    keyword TEXT UNIQUE NOT NULL)

CREATE TABLE IF NOT EXISTS paper_keyword (
    paper_id INTEGER,
    keyword_id INTEGER,
    PRIMARY KEY (paper_id, keyword_id),
    FOREIGN KEY (paper_id) REFERENCES papers(paper_id),
    FOREIGN KEY (keyword_id) REFERENCES keywords(keyword_id))
    
```





```
SELECT *
FROM papers
WHERE strftime('%Y', published_date) = '2025';

SELECT p.paper_id, p.title, p.published_date, p.abstract
FROM papers p
         JOIN paper_author pa ON p.paper_id = pa.paper_id
         JOIN authors a ON pa.author_id = a.author_id
WHERE a.name = 'Bo Shen'
ORDER BY p.published_date DESC;

SELECT p.paper_id, p.title, p.published_date
FROM papers p
         JOIN paper_keyword pk ON p.paper_id = pk.paper_id
         JOIN keywords k ON pk.keyword_id = k.keyword_id
WHERE k.keyword = 'Java'
ORDER BY p.published_date DESC;




```

